In [1]:
from torchvision import datasets
from matplotlib import pyplot as plt
import torch
from torchvision import transforms
import torch.optim as optim
import torch.nn as nn
from matplotlib import pyplot as plt
import numpy as np

data_path = 'C:/Users/keith/Desktop/School/RTAI/HW/HW4'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True) # <1>
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True) # <2>

Files already downloaded and verified
Files already downloaded and verified


In [2]:
transformed_cifar10 = datasets.CIFAR10(data_path, train=True, download=False,
                                      transform = transforms.Compose([
                                          transforms.ToTensor(),
                                          transforms.Normalize((0.4914, 0.4822, 0.4465),
                                                               (0.2470, 0.2435, 0.2616))
                                      ]))

transformed_cifar10_val = datasets.CIFAR10(data_path, train=False, download=False,
                                      transform = transforms.Compose([
                                          transforms.ToTensor(),
                                          transforms.Normalize((0.4914, 0.4822, 0.4465),
                                                               (0.2470, 0.2435, 0.2616))
                                      ]))

In [3]:
def softmax(x):
    return torch.exp(x) / torch.exp(x).sum()

In [4]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [5]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [6]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1): 
        loss_train = 0.0
        for imgs, labels in train_loader: 
            
            outputs = model(imgs)
            
            loss = loss_fn(outputs, labels) 

            optimizer.zero_grad()  
            
            loss.backward()
            
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

In [7]:
train_loader = torch.utils.data.DataLoader(transformed_cifar10, batch_size=64, shuffle=True)
model = Net()
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

import time

start = time.time()
training_loop(
    n_epochs = 200,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)
end = time.time()

ttime = end - start
print(str(ttime) + " seconds")

2021-04-21 17:51:37.096098 Epoch 1, Training loss 2.015313145602146
2021-04-21 17:54:24.752637 Epoch 10, Training loss 1.182037341777626
2021-04-21 17:57:31.533816 Epoch 20, Training loss 0.9984736375491637
2021-04-21 18:00:38.223945 Epoch 30, Training loss 0.9112453408101026
2021-04-21 18:03:42.971171 Epoch 40, Training loss 0.8569323506272967
2021-04-21 18:06:55.143347 Epoch 50, Training loss 0.8124551888164657
2021-04-21 18:10:11.599280 Epoch 60, Training loss 0.7789125465752219
2021-04-21 18:14:10.376259 Epoch 70, Training loss 0.7510147317672324
2021-04-21 18:18:16.963214 Epoch 80, Training loss 0.7276073113807937
2021-04-21 18:22:12.751148 Epoch 90, Training loss 0.706681099960871
2021-04-21 18:26:16.754904 Epoch 100, Training loss 0.6887522609642399
2021-04-21 18:30:19.706138 Epoch 110, Training loss 0.672220994596896
2021-04-21 18:33:33.177110 Epoch 120, Training loss 0.6583433915739474
2021-04-21 18:36:41.705750 Epoch 130, Training loss 0.6451079074455344
2021-04-21 18:39:54.5

In [8]:
val_loader = torch.utils.data.DataLoader(transformed_cifar10_val, batch_size=64,
                                         shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():  # <1>
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <2>
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum())  # <4>

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.78
Accuracy val: 0.63


In [9]:
numel_list = [p.numel()
              for p in model.parameters()
              if p.requires_grad == True]
sum(numel_list), numel_list

(18354, [432, 16, 1152, 8, 16384, 32, 320, 10])